# A Zindi Competition on identifying Landslides in Hong Kong

https://zindi.africa/competitions/landslide-prevention-and-innovation-challenge/

This notebook includes some exploratory data analysis (EDA) on the training data from the landslide prevention and innovation challenge.

The data includes 9 features from areas surrounding a possible landslide site. The features are:
* Elevation
* Slope
* Aspect
* Planform curvature
* Profile curvature
* Length-slope-factor
* Topographic wetness index
* Geology/Surface material (7 different materials)
* Step duration orographic intensification factor

The features are presented for 5x5 m² fields surrounding a possible landslide site in two rings. They are counted from 1 to 25, with 13 begin the centre. If a landslide occured, this landslide would be in field 13.

|  |  |  |  |  |
|:---:|:---:|:---:|:---:|:---:|
|1|6|11|16|21|
|2|7|12|17|22|
|3|8|**13**|18|23|
|4|9|14|19|24|
|5|10|15|20|25|

The EDA includes:
* Overview graphs for correlations of different locations of single features
* Distributions of the features depending on the label
* Correlations between features

In [ ]:
# import the libraries used
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Loading the data
train = pd.read_csv('data/Train.csv')

In [ ]:
# Show data
train.head()

In [ ]:
train.info()

In [ ]:
# Check for missing values
train.isna().sum().sum()
# No missing values

In [ ]:
# Check for duplicates
train.duplicated().any()
# No duplicates

In [ ]:
# Distribution of target value (0 means "no landslide", 1 means "landslide occured")
train['Label'].value_counts(normalize=True)
# unbalanced dataset has to be considered

## Overview plots for each feature

Plots showing the spatial distribution of all features to check for volatility. Some features might change in value between the different positions of the feature for a single observation. For each feature, the same 5 positions are shown for better comparison.

In [ ]:
def feature_pair_plot(feature, position, title, plotkind):
    """
    Creates a seaborn-pairplot for several positions of the same feature.
    
    feature: name of the feature-column without number and underscore
    position: list of positions of the measured fields, 13 is the central position with a possible landslide
    title: Superior title of the diagram
    plotkind: kind of plot of the comparison plots. Possible values: 'scatter', 'kde', 'hist', 'reg'
    """
    feature_cols=[]
    for i in position:
        feature_cols.append(str(i)+'_'+feature)
    plt.figure()
    sns.pairplot(train[feature_cols], kind=plotkind)
    plt.suptitle(title, y=1.01)
    plt.show()

In [ ]:
# Reminder of the spatial distribution of the features
# 1 6  11 16 21
# 2 7  12 17 22
# 3 8  13 18 23
# 4 9  14 19 24
# 5 10 15 20 25

# Corner, middle and adjescent to centre positions 
# used to plot correlations between different positions of the same feature
position = [1, 17, 13, 9, 25]

### Highly correlated features

Elevation and orographic intensification factor are highly correlated between the different positions. 

For the elevation, this means, that no high cliffs or deep fractures are part of the data. 

The orographic intensification factor has something to do with the orographic influence on rainfall. Rainfall usually doesn't change in huge amounts in such a small area (25x25 m²), which can also be seen in the data. I could not find much information about this factor. See [this paper](https://www.mdpi.com/2073-4441/12/4/1177) on a study in Taiwan for more details. It includes the SDOIF.

In [ ]:
title='Digital elevation of the terrain surface in meter'
feature_pair_plot(feature='elevation', position=position, title=title, plotkind='hist')

# perfect correlation between different locations

In [ ]:
title='Step duration orographic intensification factor'
feature_pair_plot(feature='sdoif', position=position, title=title, plotkind='hist')

# perfect correlation all over

### Little correlation with far positions

In [ ]:
title='Angle of the slope inclination in degree'
feature_pair_plot(feature='slope', position=position, title=title, plotkind='hist')

# Slope varies a lot in these 25 x 25 m² samples
# Positions closer together vary less

In [ ]:
title='Exposition of the slope in degree'
feature_pair_plot(feature='aspect', position=position, title=title, plotkind='hist')

# High values in far edges due to circular behavior of the feature. 
# Tansformation of the feature is needed to take this into account for machine learning algorithms.
# Most aspects are close to each other though

In [ ]:
title='Length-slope factor'
feature_pair_plot(feature='lsfactor', position=position, title=title, plotkind='hist')

# some correlations throughout, but a lot more for close positions

In [ ]:
title='Topographic wetness index'
feature_pair_plot(feature='twi', position=position, title=title, plotkind='hist')

# Highly skewed
# Logarithmic transformation should help

### No correlation with far positions

**Planform and profile curvatures**

Distributions: 
Single positions show normal distributions. They are not skewed and centered around zero. 

Correlations: 
While positions close together show a positive correlation, this reduces very quickly as the distance increases. The farthest positions don't show a correlation at all as the scatterplot only shows a circle.

In [ ]:
title='Planform curvature'
feature_pair_plot(feature='placurv', position=position, title=title, plotkind='hist')

# no correlation between far away positions
# positions closer together are still quite similar

In [ ]:
title='Profile curvature'
feature_pair_plot(feature='procurv', position=position, title=title, plotkind='hist')

# no correlation between far away positions
# positions closer together are still quite similar

## Geology

The surface materials are:
1. Weathered Cretaceous granitic rocks
2. Weathered Jurassic granite rocks
3. Weathered Jurassic tuff and lava
4. Weathered Cretaceous tuff and lava
5. Quaternary deposits
6. Fill
7. Weathered Jurassic sandstone, siltstone and mudstone

### Distribution

The differences in counts for each position are compared for each material.
Due to huge differences in occurences from 19 for material 6 and 6166 for material 3, only the statistical values are shown for each material. 

By far the most common surface material is jurassic tuff and lava. Fill and weathered Cretaceous tuff and lava only occur very rarely. Overall the distribution of surface materials show only very little variation.

In [ ]:
geology = pd.DataFrame()
for i in range(25):
    geology[f'{i+1}_geology'] = train[f'{i+1}_geology'].value_counts()
geology.T.describe()

### Correlation between central position 13 and position 1 

Mostly the material stays the same per observation. 
For material 5, sometimes the material in different positions change to material 2 or 3. Materials 2 and 3 belong to the Jurassic age, which is the oldest age mentioned here. These old rocks might therefore have been covered by the deposits from the Quaternary age.

The most common surface material is material 3 (Jurassic tuff and lava)

In [ ]:
# inspired by https://stackoverflow.com/questions/60119971/seaborn-matplotlib-categorical-plot-markers-size-by-count-of-observations

# the categorical variables to plot
x = '1_geology'
y = '13_geology'

# Compute the counts of observations
df_counts = train.groupby([x, y]).size().reset_index()
df_counts.columns.values[df_counts.columns == 0] = 'count'

# Compute a size variable for the markers so that they have a good size regardless
# of the total count and the number of unique values in each categorical variable
scale = 200*df_counts['count'].size
size = df_counts['count']/df_counts['count'].sum()*scale

# Create matplotlib scatter plot with additional formatting
fig, ax = plt.subplots(figsize=(5,4))
ax.scatter(x, y, size, data=df_counts)
ax.set_xlabel('1_geology')
ax.set_ylabel('13_geology')
plt.show()

## Dependence of target on feature distribution

| feature | dependence |
| --- | --- |
| elevation | more landslides at higher elevation |
| slope | more landslides at higher slopes |
| aspect | more landslides between 150 and 250 ° |
| placurv | no difference |
| procurv | no difference |
| lsfactor | more landslides at higher lsfactors |
| twi | more landslides at lower twi |
| geology |  |
| sdoif | hardly any landslides below 1.25 |

In [ ]:
# Calculating relative values of geology for better visibility
geology = train.value_counts(subset=['13_geology', 'Label'], sort=False).reset_index(name="count")
geology_sum = train.value_counts('13_geology', sort=False).reset_index(name="sum")
geology_full = geology.merge(geology_sum, on='13_geology')
geology_full['rel'] = geology_full['count']/geology_full['sum']
geology_full

In [ ]:
fig, axs = plt.subplots(3, 3, figsize = (13, 10))

sns.histplot(data=train, x="13_elevation", hue="Label", ax=axs[0,0], element="step", fill=False)
sns.histplot(train, x="13_slope", hue="Label", ax=axs[0,1], element="step", fill=False, legend=False)
sns.histplot(train, x="13_aspect", hue="Label", ax=axs[0,2], element="step", fill=False, legend=False)
sns.histplot(train, x="13_placurv", hue="Label", ax=axs[1,0], element="step", fill=False, legend=False)
sns.histplot(train, x="13_procurv", hue="Label", ax=axs[1,1], element="step", fill=False, legend=False)
sns.histplot(train, x="13_lsfactor", hue="Label", ax=axs[1,2], element="step", fill=False, legend=False)
sns.histplot(train, x="13_twi", hue="Label", ax=axs[2,0], element="step", fill=False, legend=False, log_scale=True)
sns.histplot(train, x="13_sdoif", hue="Label", ax=axs[2,1], element="step", fill=False, legend=False)
sns.histplot(geology_full, x="13_geology", hue="Label", weights='rel', bins=7, ax=axs[2,2], multiple="stack", fill=True, legend=False)

axs[2,2].set_ylabel('percentage')

fig.tight_layout()
plt.show()

## Correlations

High sdoifs can be at high elevations. Sdoif usually has higher values at higher elevations.

In [ ]:
cols = ['1_elevation', '1_slope', '1_aspect', '1_placurv', '1_procurv', 
        '1_lsfactor', '1_twi', '1_geology', '1_sdoif']
sns.pairplot(train[cols])

# cretaceus tuff and lava only in very high areas, fill and sandstone, siltstone and mudstone only in low areas
# little correlation between most features

In [ ]:
# Quantify correlations
corr = train[cols].corr()
plt.figure(figsize = (13, 8))
sns.heatmap(corr, cmap='RdYlGn', annot = True, center = 0)
plt.title('Correlogram', fontsize = 15, color = 'darkgreen')
plt.show()

# highest correlation between slope and lsfactor, procurv and placurv, placurv and twi